In [30]:
import requests
import pandas as pd
from selenium import webdriver
from scrapy.http import TextResponse
import seaborn as sns
import getpass
import time

In [31]:
# 클래스로 모듈화

In [32]:
# %%writefile insta_crawling.py
# import requests
# import pandas as pd
# from selenium import webdriver
# from scrapy.http import TextResponse
# import getpass
# import time


class instagram_crawling():
    
    driver = ""
    
    def __init__(self):
        login_url = "https://www.instagram.com/accounts/login/?source=auth_switcher"
        self.driver = webdriver.Chrome()
        self.driver.get(login_url)
    
    
    # 로그인 함수 (seleniun webdriver)
    def login(self):

        my_id = input('id를 입력하세요: ')
        my_password = getpass.getpass('password를 입력하세요: ')

        self.driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP[type=text]').send_keys(my_id)
        self.driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP[type=password]').send_keys(my_password)
        self.driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF[type=submit]').click()

        time.sleep(3)

        # 알림설정 하라는 modal창이 뜨는경우 '나중에 하기'를 클릭하는 코드
        alert_modal = self.driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div')
        if alert_modal:
            self.driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div > div.mt3GC > button.aOOlW.HoLwm').click()




    # 검색어를 입력하고 게시물이 가장 많은 검색결과를 클릭 (seleniun webdriver)
    def crawling_keyword(self, word):
        keyword = word

        self.driver.find_element_by_css_selector('#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > input').send_keys(keyword)

        # 키워드 입력하고, 키워드에 대한 검색 리스트가 뜨는데 걸리는 시간만큼 기다려준다
        time.sleep(1.5)

        search_list = self.driver.find_elements_by_css_selector('#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > div:nth-child(4) > div.drKGC > div > a')

        # 서칭된 a 엘리먼트들 중에 게시물 갯수가 가장 많은 a 엘리먼트를 뽑는다
        number_list = []

        for element in search_list:
            splited_element = element.text.split('게시물')
            if len(splited_element) == 2:
                number_list.append(int(re.sub(",", "", splited_element[1])))
            else:
                number_list.append(0)

        max_number = max(number_list)
        click_index = number_list.index(max_number)

        # 클릭하고 렌더링이 완료되도록 조금 기다리기
        search_list[click_index].click()
        time.sleep(2.5)




    # 처음 렌더링된 화면의 24개의 게시물을 크롤링하는 함수
    def initial_crawling(self):
        urls = self.driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
        urls = [url.get_attribute("href") for url in urls]

        hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

        for i in range(len(urls)):
            req = requests.get(urls[i])
            response = TextResponse(req.url, body=req.text, encoding="utf-8")
            hash_tag_list = response.css("meta[property='instapp:hashtags']")
            hash_tag_list = ['#' + hash_tag.attrib['content'] for hash_tag in hash_tag_list]
            hash_tags = ', '.join(hash_tag_list)

            data = [
                {'time': 0, 'hash_tag': hash_tags}
            ]
            df = pd.DataFrame(data)
            hash_tag_df = hash_tag_df.append(df)

        return hash_tag_df.reset_index().drop(columns=['index'])




    # 스크롤내리면서 12개씩 추가되는 게시물을 크롤링하는 함수 (두번째만 실행) - chrome 전체화면 기준
    # 현재 브라우저 화면 크기에 따라 추가적으로 불러오는 div 갯수가 달라짐
    # 전체화면 기준 div의 urls 총 갯수 = 24 -> 36 -> 45 -> 45 -> ...
    def second_crawling(self):
        
        hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])
        
        for i in range(0, 1):
            self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            time.sleep(1.5)

            urls = self.driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
            urls = [url.get_attribute("href") for url in urls]

            for i in range(len(urls) - 12, len(urls)):
                req = requests.get(urls[i])
                response = TextResponse(req.url, body=req.text, encoding="utf-8")
                hash_tag_list = response.css("meta[property='instapp:hashtags']")
                hash_tag_list = ['#' + hash_tag.attrib['content'] for hash_tag in hash_tag_list]
                hash_tags = ', '.join(hash_tag_list)

                data = [
                    {'time': 0, 'hash_tag': hash_tags}
                ]
                df = pd.DataFrame(data)
                hash_tag_df = hash_tag_df.append(df)

        return hash_tag_df.reset_index().drop(columns=['index'])



    # 세번째부터는 이 함수 반복하기 - 게시물 9개씩 크롤링
    def repeat_crawling(self):
        
        hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])
        
        for i in range(0, 1):
            self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            time.sleep(1.5)

            urls = self.driver.find_elements_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
            urls = [url.get_attribute("href") for url in urls]

            for i in range(len(urls) - 9, len(urls)):
                req = requests.get(urls[i])
                response = TextResponse(req.url, body=req.text, encoding="utf-8")
                hash_tag_list = response.css("meta[property='instapp:hashtags']")
                hash_tag_list = ['#' + hash_tag.attrib['content'] for hash_tag in hash_tag_list]
                hash_tags = ', '.join(hash_tag_list)

                data = [
                    {'time': 0, 'hash_tag': hash_tags}
                ]
                df = pd.DataFrame(data)
                hash_tag_df = hash_tag_df.append(df)

        return hash_tag_df.reset_index().drop(columns=['index'])
    

    
# instagram_crawling 클래스를 이용해서 해시태그를 크롤링하고 전체 데이터프레임을 리턴한다
def crawling_start(keyword, repeat_num):
    
    keyword = keyword
    num = repeat_num
    result_df = pd.DataFrame(columns=['time', 'hash_tag'])
    
    insta = instagram_crawling()
    
    insta.login()
    insta.crawling_keyword(keyword)
    
    initial_df = insta.initial_crawling()
    result_df = pd.concat([result_df, initial_df])
    
    second_df = insta.second_crawling()
    result_df = pd.concat([result_df, second_df])
    
    # 마지막 크롤링 함수 반복
    for i in range(0, num):
        try:
            df = insta.repeat_crawling()
            result_df = pd.concat([result_df, df])
        except Exception as exc:
            print('에러가 발생했습니다 : ', exc)
    
    return result_df.reset_index().drop(columns=['index'])
    


In [36]:
# 데이터 row 갯수 = 24 + 12 + (9 * 10)

df = crawling_start('초코우유', 10)

id를 입력하세요: yeejun90
password를 입력하세요: ········


In [37]:
len(df)

126

In [40]:
df

,time,hash_tag
0,0,#초코우유
1,0,"#애월해안도로맛집, #제주카페투어, #외도339, #제주시카페, #라떼아트, #ba..."
2,0,"#서울숲, #핵잼, #홍대카페, #바리스타, #홍차, #달콤, #도산공원, #간식스..."
3,0,"#베이킹, #단체간식, #리얼초코우유, #당충전, #커피맛집, #울산, #럽스타그램..."
4,0,"#산딸기요거트, #오렌지요거트, #오레오, #제주말차, #카라멜마끼아또, #소금바닐..."
...,...,...
121,0,"#베어벨스, #레시피노트, #프로틴음료, #요리하는남자, #일상에서프로틴, #팬케이..."
122,0,"#라인일러스트, #그림, #digital, #디지털, #line, #드로잉, #초코..."
123,0,"#art, #일러스트, #라인일러스트, #초코우유, #디지털그림, #drawing,..."
124,0,"#스누피초코우유, #충전, #공부난장판, #영어공부, #공부, #난장판, #힘내자,..."
